In [4]:
import pandas as pd
import numpy as np
from datetime import date


def cb_curr():
    day = date.today()
    today = day.strftime('%d/%m/%Y')

    dict_of_currencies = {
        'R01235': 'USD',
        'R01239': 'EUR',
        'R01010': 'Australian_Dollar',
        'R01375': 'China_Yuan',
        'R01035': 'British_Pound',
        'R01335': 'Kazakhstan_Tenge',
        'R01815': 'Japanese_Yen',
        'R01775': 'Swiss_Franc'
    }

    list_of_currencies = [x for x in dict_of_currencies.keys()]

    URL_list = []
    for currency in list_of_currencies:
        URL = f'http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={today}&date_req2={today}&VAL_NM_RQ={currency}'
        URL_list.append(URL)

    currency_df = pd.read_excel(
        '/Users/hotdonkey/Documents/GitHub/data_science_learning/code/LME/parser_beta/data/cb_curr.xlsx', index_col=0)

    for url_element in URL_list:
        response_df = pd.read_xml(url_element)
        response_df['Date'] = pd.to_datetime(
            response_df['Date'], dayfirst=True)
        response_df['Value'] = response_df['Value'].apply(
            lambda x: x.replace(',', '.'))
        response_df['Value'] = response_df['Value'].apply(pd.to_numeric)
        response_df = response_df.replace(dict_of_currencies)
        currency_df = pd.concat(
            [currency_df, response_df], axis=0, ignore_index=True)
        currency_df.drop_duplicates(inplace=True)

    with pd.ExcelWriter(
            "/Users/hotdonkey/Documents/GitHub/data_science_learning/code/LME/parser_beta/data/cb_curr.xlsx") as writer:
        currency_df.to_excel(writer, sheet_name='curr')

    return currency_df


if __name__ == '__main__':
    cb_curr()
